In [1]:
import pandas as pd
import numpy as np
import seaborn as sns
import matplotlib.pyplot as plt
import tensorflow as tf
import tensorflow.python.keras as krs
from keras.layers import LSTM, Dropout, Dense
from keras.models import Sequential

2022-10-15 17:11:06.143801: I tensorflow/core/platform/cpu_feature_guard.cc:193] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical operations:  AVX2 FMA
To enable them in other operations, rebuild TensorFlow with the appropriate compiler flags.
2022-10-15 17:11:06.285987: W tensorflow/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'libcudart.so.11.0'; dlerror: libcudart.so.11.0: cannot open shared object file: No such file or directory
2022-10-15 17:11:06.286013: I tensorflow/stream_executor/cuda/cudart_stub.cc:29] Ignore above cudart dlerror if you do not have a GPU set up on your machine.
2022-10-15 17:11:06.322425: E tensorflow/stream_executor/cuda/cuda_blas.cc:2981] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered
2022-10-15 17:11:07.287699: W tensorflow/stream_executor/platform/de

In [2]:
_df = pd.read_csv("Data/wig20_d.csv")
# df = pd.read_csv("Data/mwig40_d.csv")
# df = pd.read_csv("Data/swig80_d.csv")

_df.drop(range(1000), inplace=True)
_df.reset_index(inplace=True, drop=True)
_df.head()

,Data,Otwarcie,Najwyzszy,Najnizszy,Zamkniecie,Wolumen
0,1997-02-06,1717.2,1779.8,1717.2,1779.8,4435886.0
1,1997-02-07,1778.2,1808.7,1778.2,1799.0,5448243.0
2,1997-02-10,1797.4,1797.4,1780.3,1783.3,6513315.0
3,1997-02-11,1803.3,1832.2,1803.3,1832.0,5146340.0
4,1997-02-12,1824.6,1824.6,1804.9,1810.2,5748398.0


In [3]:
_df.head()

,Data,Otwarcie,Najwyzszy,Najnizszy,Zamkniecie,Wolumen
0,1997-02-06,1717.2,1779.8,1717.2,1779.8,4435886.0
1,1997-02-07,1778.2,1808.7,1778.2,1799.0,5448243.0
2,1997-02-10,1797.4,1797.4,1780.3,1783.3,6513315.0
3,1997-02-11,1803.3,1832.2,1803.3,1832.0,5146340.0
4,1997-02-12,1824.6,1824.6,1804.9,1810.2,5748398.0


In [4]:
_data_len = len(_df.index)
_train_set_ratio, _test_set_ratio, _validation_set_ratio = .8, .1, .1

_train_set_len = int(_data_len * _train_set_ratio)
_test_set_len = int(_data_len * _test_set_ratio)
_validation_set_len = int(_data_len * _validation_set_ratio)

_train_set_end = _train_set_len
_test_set_end = _train_set_end + _test_set_len
_validation_set_end = _test_set_end + _validation_set_len

#assert _train_set_ratio + _test_set_ratio + _validation_set_ratio == 1

_data = _df.to_numpy().transpose()
colnames = _df.columns.to_numpy()

#train_set, test_set, validation_set = _df.iloc[:_train_set_end, :], _df.iloc[_train_set_end:_test_set_end, :], _df.iloc[_test_set_end:_validation_set_end, :]
train_set, test_set, validation_set = _data[:, :_train_set_end], _data[:, _train_set_end:_test_set_end], _data[:, _test_set_end:_validation_set_end]


In [5]:
colnames

array(['Data', 'Otwarcie', 'Najwyzszy', 'Najnizszy', 'Zamkniecie',
       'Wolumen'], dtype=object)

In [6]:
train_set[:, :10]

array([['1997-02-06', '1997-02-07', '1997-02-10', '1997-02-11',
        '1997-02-12', '1997-02-13', '1997-02-14', '1997-02-17',
        '1997-02-18', '1997-02-19'],
       [1717.2, 1778.2, 1797.4, 1803.3, 1824.6, 1843.4, 1883.0, 1874.1,
        1894.9, 1880.7],
       [1779.8, 1808.7, 1797.4, 1832.2, 1824.6, 1888.3, 1898.3, 1897.6,
        1908.8, 1892.5],
       [1717.2, 1778.2, 1780.3, 1803.3, 1804.9, 1843.4, 1870.2, 1874.1,
        1885.7, 1876.2],
       [1779.8, 1799.0, 1783.3, 1832.0, 1810.2, 1887.9, 1870.2, 1891.4,
        1885.7, 1880.1],
       [4435886.0, 5448243.0, 6513315.0, 5146340.0, 5748398.0, 6054225.0,
        6470972.0, 5876872.0, 5494772.0, 6051072.0]], dtype=object)

In [7]:
test_set[:, :10]

array([['2017-08-23', '2017-08-24', '2017-08-25', '2017-08-28',
        '2017-08-29', '2017-08-30', '2017-08-31', '2017-09-01',
        '2017-09-04', '2017-09-05'],
       [2387.52, 2395.3, 2464.81, 2474.13, 2525.41, 2507.29, 2520.23,
        2512.46, 2513.38, 2521.04],
       [2395.62, 2456.82, 2481.63, 2534.04, 2525.41, 2537.24, 2532.83,
        2528.82, 2525.72, 2550.74],
       [2374.66, 2395.3, 2459.27, 2470.06, 2479.35, 2503.35, 2501.59,
        2508.86, 2508.27, 2513.24],
       [2391.06, 2456.27, 2471.74, 2531.31, 2497.58, 2524.57, 2516.26,
        2527.62, 2519.42, 2517.45],
       [14965936.0, 28821793.0, 19644957.0, 21492576.0, 23903291.0,
        24313159.0, 26150711.0, 15899558.0, 15063183.0, 20397129.0]],
      dtype=object)

In [8]:
validation_set[:, :10]

array([['2020-03-24', '2020-03-25', '2020-03-26', '2020-03-27',
        '2020-03-30', '2020-03-31', '2020-04-01', '2020-04-02',
        '2020-04-03', '2020-04-06'],
       [1449.34, 1487.75, 1431.53, 1494.31, 1482.1, 1486.89, 1495.43,
        1490.41, 1492.1, 1534.81],
       [1473.02, 1505.79, 1492.87, 1499.62, 1484.62, 1523.49, 1500.43,
        1501.33, 1509.07, 1585.04],
       [1424.14, 1413.22, 1410.91, 1465.48, 1443.16, 1484.33, 1473.64,
        1464.86, 1474.44, 1534.81],
       [1451.02, 1441.83, 1492.01, 1475.9, 1477.65, 1512.84, 1482.42,
        1491.55, 1506.48, 1581.05],
       [37402637.0, 47382329.0, 36791461.0, 33221292.0, 28425038.0,
        59768681.0, 36501160.0, 48115276.0, 49706290.0, 45607978.0]],
      dtype=object)

In [9]:
train_set.shape

(6, 5143)

In [10]:
def batches_create(data_array):
    _x_train = [data_array[:, i:i+60] for i in range(data_array.shape[1]-60-1)]
    _y_train = [data_array[:, i+60] for i in range(data_array.shape[1]-60-1)]
    return _x_train, _y_train


In [11]:
X_train, y_train = batches_create(train_set)